In [1]:
from pathlib import Path
import sys, os
path = Path(os.path.dirname(os.path.abspath('')))
if path not in sys.path:
    sys.path.append(os.path.join(path))
    
from koebe.datastructures.dcel import *

In [9]:
class Tiling(DCEL):
    
    def __init__(self, outerFaceData = None):
        super().__init__(outerFaceData)

        self.dartLevels = []
        self.edgeLevels = []
        self.faceLevels = []
        self.subdivisionLevel = 1

    def addLevel(self):
        old_darts, old_edges, old_faces = self.darts, self.edges, self.faces
        
        self.dartLevels.append(self.darts)
        self.edgeLevels.append(self.edges)
        self.faceLevels.append(self.faces)
        
        self.darts = []
        self.edges = []
        self.faces = []
        
        self.subdivisionLevel += 1
        
        return old_darts, old_edges, old_faces

        
tiling = Tiling.generateCycle(vdata = ["A","B","C","D","E","F","G","H"])
tile = tiling.faces[-1]
tile.tileType = "chair"
#tile.starTriangulate(vdata = "z")

class TilingRules:
    
    def __init__(self):
        self.prototiles = {}
    
    def createPrototile(self, tileType, tileVerts):
        self.prototiles[tileType] = Prototile(self, tileType, tileVerts)
        return self.prototiles[tileType]
    
    def getPrototile(self, tileType):
        return self.prototiles[tileType]
    
    def createInitialTile(self, tileType):
        prototile = self.getPrototile(tileType).tileVerts
        tiling    = DCEL.generateCycle(prototile.tileVerts)
        tiling.edgeStack = []
        tiling.dartStack = []
        tiling.faceStack = []
        tiling.faces[1].tileType = tileType
        tiling.tilingLevel = 1
        return tiling
    
class Prototile:
    
    def __init__(self, tilingRules: TilingRules, tileType: str, tileVerts: str):
        """
        Args: 
            tilingRules: The TilingRules object that this prototile is a part of. 
            tileVerts: The name of the vertices. For example, "ABCDEF" is a 
                six sided tile with vertex names A, B, C, D, E, and F. 
        """
        self.tilingRules  = tilingRules
        self.tileType     = tileType
        self.tileVerts    = tileVerts
        self.splitRules   = dict()
        self.newVertRules = list()
        self.subtiles     = list()
    
    def addSplitEdgeRule(self, edge, newverts):
        """
        Adds a rule to split an edge by introducing a series of new verts. For example
        if edge is "AB" and newverts is "a" this command will split the edge AB into
        Aa and aB. If newverts is "abc" then AB is split into Aa ab bc and cB. 
        Args:
            self
            edge: The name of the edge to split, like "AB"
            newverts: The new vertices to introduce, like "a". 
        """
        self.splitRules[edge] = newverts
    
    def addSplitEdgeRules(self, splitCommands):
        """
        Args: 
            self
            splitCommands: A list of tuples of the form (edge, newverts) that directs
                the tiling to split this edge by introducing a series of new vertices. 
                For example, ("AB", "a") would split the edge AB into two edges Aa and 
                aB. 
        """
        for edge, newverts in splitCommands: 
            self.addSplitEdgeRule(edge, newverts)
    
    def addNewVertexRule(self, vertName):
        self.newVertRules.append(vertName)
    
    def addNewVertexRules(self, vertNames):
        for vertName in vertNames:
            self.addNewVertexRule(vertName)
    
    def addSubtile(self, subtileType, subtileVerts):
        self.subtiles.append((subtileType, subtileVerts))
    
class PrototileFormationError(Exception):
    pass

# The chair tiling only has one prototile
rules = TilingRules()
chair = rules.createPrototile("chair", ["A","B","C","D","E","F","G","H"])

# Edges that need to be split
chair.addSplitEdgeRules(((("A","B"), "a"), (("B","C"), "b"), (("C","D"), "c"),
                        (("D","E"), "d"), (("E","F"), "e"), (("F","G"), "f"),
                        (("G","H"), "g"), (("H","A"), "h")))

# New vertices to create
chair.addNewVertexRules(("i","j","k","l","m"))

# The subdivision subtiles: 
chair.addSubtile("chair", ("A","h","H","l","k","j","B","a"))
chair.addSubtile("chair", ("G","f","F","e","m","l","H","g"))
chair.addSubtile("chair", ("k","l","m","e","E","d","i","j"))
chair.addSubtile("chair", ("C","b","B","j","i","d","D","c"))

    
def tilingPass(tileRules, tiling):
    
    tiles = [tile for tile in tiling.faces if tile != tiling.outerFace]
    
    # Annotate each tile with dictionaries to switch between 
    # vertex names and darts within the tile and edge names and darts. 
    for tile in tiles: 
        prototile = tileRules.getPrototile(tile.tileType)
        theDarts  = tile.darts()
        
        if len(theDarts) != len(prototile.tileVerts):
            raise PrototileFormationError(
                    f"Prototile vertex count ({len(theDarts)}) does not match tile"
                    + f" vertex count for tile type {tile.tileType}."
            )
        
        tile.vertexByName = {}
        # Build a mapping from vertex names to darts originating at the vertex
        # and vice versa. 
#         tile.vertexToDart   = {}
        for i in range(len(theDarts)):
            vertexName = prototile.tileVerts[i]
#             tile.vertexToDart[vertexName] = theDarts[i]
            theDarts[i].originName = vertexName
        
        tile.edgeNameToDart = {}
        for dart in theDarts:
            name = (dart.originName, dart.next.originName)
            tile.edgeNameToDart[name] = dart
            dart.edgeName = name
        
        tile.splitEdgeNameToDart = {}
            
    # For each tile, annotate any edge that needs to be split with its split rules. 
    for tile in tiles:
        prototile = tileRules.getPrototile(tile.tileType)
        theDarts  = tile.darts()
        for dart in theDarts:
            if dart.edgeName in prototile.splitRules:
                dart.splitRule = prototile.splitRules[dart.edgeName]
            else:
                dart.splitRule = ""
    for dart in tiling.outerFace.darts():
        dart.splitRule = ""
    
    def annotateAndCheckEdge(edge):
        dart1, dart2 = edge.darts()
        if dart1.face == edge.dcel.outerFace:
            edge.newVertexCount = len(dart2.splitRule)
        elif dart2.face == edge.dcel.outerFace:
            edge.newVertexCount = len(dart1.splitRule)
        elif len(dart1.splitRule) == len(dart2.splitRule):
            edge.newVertexCount = len(dart2.splitRule)
        else:
            edge.newVertexCount = None
        return edge.newVertexCount != None
    
    def raiseEdgeError(edge):
        dart1, dart2 = edge.darts()
        tileName1 = dart1.face.tileType
        tileName2 = dart2.face.tileType
        splitCount1 = len(dart1.splitRule) + 1
        splitCount2 = len(dart2.splitRule) + 1
        
        raise PrototileFormationError(
                "Edge subdivision cannot be performed because "
                + f"edge {dart1.edgeName} of tile {tileName1}"
                + f" has to split into {splitCount1} edges"
                + " but is matched with "
                + f"edge {dart2.edgeName} of tile {tileName2}," 
                + f" which has to split into {splitCount2} edges"
        )
    
    # Check that each edge compiles:
    for edge in tiling.edges:
        if not annotateAndCheckEdge(edge):
            raiseEdgeError(edge)
    
    # Now we need to actually split each edge introducing new vertices
    # then add the new edges and stitch the tiles up. 

    def getSplitVertexNames(dart):
        """ For each dart get the vertex names for the split. For example
            if we're splitting a dart representing AB in a tiling by introducing
            a single vertex named a, then getSplitVertexNames will return ("A", "a", "B")
            as a tuple.
            
            Args: 
            dart: The dart that needs to be split. 
            
            Returns:
            A tuple containing the vertex names for the split version of the dart.
        """
        if dart.face != dart.dcel.outerFace:
            split_vertex_names = (dart.edgeName[0]) + dart.splitRule + (dart.edgeName[-1])
#             print(f"dart splitRule: {dart.edgeName} to {dart.splitRule} to form {split_vertex_names}")
        else:
            split_vertex_names = None
        return split_vertex_names
    
    def setVertexAndEdgeNames(dartList, vertexNames, parentTile, lastVertex):
        if vertexNames != None:
            for i in range(len(dartList)):
                d = dartList[i]
                d.edgeName = (vertexNames[i], vertexNames[i+1])
                if parentTile != d.dcel.outerFace:
                    parentTile.vertexByName[vertexNames[i]] = d.origin 
                    parentTile.splitEdgeNameToDart[d.edgeName] = d
            if parentTile != d.dcel.outerFace:
                parentTile.vertexByName[vertexNames[-1]] = lastVertex
    
    old_darts, old_edges, old_faces = tiling.addLevel() # Push all the current level info onto the various stacks and reset for new one. 
    
    # Split all edges/darts according to their split rules and store a mapping from new split dart names
    # to Dart objects in the corresponding tile. 
    for e in old_edges:
        
        # For each dart get the vertex names for the split. For example
        # if we're splitting a dart representing AB in a tiling by introducing
        # a single vertex named a, then getSplitVertexNames will return ("A", "a", "B")
        # as a tuple. 
        vertNames1 = getSplitVertexNames(e.aDart)
        vertNames2 = getSplitVertexNames(e.aDart.twin)
        
        # Create the new vertices along the edge and the new darts. 
        newVerts = [Vertex(e.dcel) for _ in range(e.newVertexCount)]  
        
        nDarts1 = ([Dart(e.dcel, origin = e.aDart.origin)] 
                   + [Dart(e.dcel, origin = v) for v in newVerts])
        
        nDarts2 = ([Dart(e.dcel, origin = e.aDart.dest)] 
                   + [Dart(e.dcel, origin = v) for v in reversed(newVerts)])
        
        setVertexAndEdgeNames(nDarts1, vertNames1, e.aDart.face, e.aDart.dest)
        setVertexAndEdgeNames(nDarts2, vertNames2, e.aDart.twin.face, e.aDart.origin)
        
        e.aDart.childDarts = nDarts1
        e.aDart.twin.childDarts = nDarts2
        
        e.childEdges = []
        for i in range(len(nDarts1)):
            nDarts1[i].makeTwin(nDarts2[len(nDarts2) - 1 - i])
            newEdge = Edge(e.dcel, aDart = nDarts1[i])
            e.childEdges.append(newEdge)
            nDarts1[i].edge = newEdge
            nDarts2[i].edge = newEdge
    
    # Split all tiles
    for tile in tiles:
        # Create any new vertices not obtained via splitting
        # and add them to the vertexByName map. 
        
        # Create a new face for each subtile getting either a dart
        # created in the edge split pass or creating a brand new dart
        
        # Loop over all the darts and find any that do not have a twin
        # for any that do not have a twin, set their twin correctly and
        # create an Edge between them
        
        pass
    print(tile.vertexByName)
        
    # Stitch up the outer face's boundary
        
#         if vertNames1:
#             for i in range(len(nDarts1)):
#                 nDarts1[i].edgeName = (vertNames1[i], vertNames1[i+1])
                
#         if vertNames2:
#             for i in range(len(nDarts2)):
#                 nDarts2[i].edgeName = (vertNames2[i], vertNames2[i+1])
                
                
#         e.aDart.children = nDarts1
#         e.aDart.twin.children = nDarts2
    


def starTriangulateAllFaces():
    global tiling
    for face in tuple(tiling.faces):
        if face != tiling.outerFace:
            face.starTriangulate()
        
# for e in tiling.edges:
#     e.level = 0
# for i in range(4):
#     edgeCount = len(tiling.edges)
#     starTriangulateAllFaces()
#     for e in tiling.edges[edgeCount:]:
#         e.level = i+1


tiling.faces[1].tileType = "chair"
tilingPass(rules, tiling)
starTriangulateAllFaces()

{'A': <koebe.datastructures.dcel.Vertex object at 0x1114da860>, 'a': <koebe.datastructures.dcel.Vertex object at 0x11148c320>, 'B': <koebe.datastructures.dcel.Vertex object at 0x1114da8d0>, 'b': <koebe.datastructures.dcel.Vertex object at 0x11148c2e8>, 'C': <koebe.datastructures.dcel.Vertex object at 0x1114da908>, 'c': <koebe.datastructures.dcel.Vertex object at 0x11150b208>, 'D': <koebe.datastructures.dcel.Vertex object at 0x1114da940>, 'd': <koebe.datastructures.dcel.Vertex object at 0x11150b4e0>, 'E': <koebe.datastructures.dcel.Vertex object at 0x1114da978>, 'e': <koebe.datastructures.dcel.Vertex object at 0x11150b668>, 'F': <koebe.datastructures.dcel.Vertex object at 0x1114da9b0>, 'f': <koebe.datastructures.dcel.Vertex object at 0x11150b7f0>, 'G': <koebe.datastructures.dcel.Vertex object at 0x1114da9e8>, 'g': <koebe.datastructures.dcel.Vertex object at 0x11150b978>, 'H': <koebe.datastructures.dcel.Vertex object at 0x1114daa20>, 'h': <koebe.datastructures.dcel.Vertex object at 0x111

In [10]:
from koebe.algorithms.hypPacker import *
packing, _ = maximalPacking(
    tiling, 
    num_passes=1000
)

KeyError: None

In [4]:
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
viewer = PoincareDiskViewer(300, 300)
viewer.addAll(packing.verts)
viewer.show()

<IPython.core.display.Javascript object>

NameError: name 'packing' is not defined

In [6]:
from koebe.geometries.euclidean2 import PointE2, SegmentE2
viewer = PoincareDiskViewer(600, 600)
edgeSegs = [SegmentE2(PointE2(e.aDart.origin.data.center.coord.real, e.aDart.origin.data.center.coord.imag), 
                      PointE2(e.aDart.dest.data.center.coord.real, e.aDart.dest.data.center.coord.imag))
           for e in packing.edges]
viewer.addAll(edgeSegs)
blue = makeStyle(stroke = "#79f", strokeWeight=1.0)
red = makeStyle(stroke = "#f97", strokeWeight=1.0)
green = makeStyle(stroke = "#7f9", strokeWeight=1.0)
gray = makeStyle(stroke = "#999", strokeWeight=1.0)
for i in range(len(edgeSegs)):
    e = edgeSegs[i]
    if tiling.edges[i].level % 4 == 0:
        viewer.setStyle(e, blue)
    elif tiling.edges[i].level % 4 == 1:
        viewer.setStyle(e, red)
    elif tiling.edges[i].level % 4 == 2:
        viewer.setStyle(e, green)
    else:
        viewer.setStyle(e, gray)
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#000…

In [7]:
print(len(tiling.verts))

329
